In [121]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn import linear_model, feature_selection
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Imputer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB

%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [122]:
crime = pd.read_csv('offenses_by_city_2013.csv',
                    header=0,
                    names=['City', 'Population', 'ViolentCrime', 'Manslaughter', 'RapeCurrent', 'RapeLegacy', 'Robbery', 'AggravatedAssault', 'PropertyCrime', 'Burglary', 'LarcenyTheft', 'MotorVehicleTheft', 'Arson3'],
                    )
crime = crime.drop(['City', 'Manslaughter', 'PropertyCrime', 'ViolentCrime', 'RapeCurrent'], axis=1)
crime.head(10)

,Population,RapeLegacy,Robbery,AggravatedAssault,Burglary,LarcenyTheft,MotorVehicleTheft,Arson3
0,"1,861",0,0,0,2,10,0,0.000
1,"2,577",0,0,3,3,20,1,0.000
2,"2,846",0,0,3,1,15,0,0.000
3,"97,956",30,227,526,705,"3,243",142,nan
4,"6,388",3,4,16,53,165,5,nan
5,"4,089",0,3,2,10,36,0,nan
6,"1,781",0,0,3,0,10,0,0.000
7,"118,296",7,31,68,204,"1,882",32,3.000
8,"9,519",2,4,3,16,188,6,1.000
9,"18,182",0,12,18,99,291,15,0.000


In [123]:
for col in crime.columns[:-1]:
    crime[col] = crime[col].str.replace(",","")

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(crime)
crime = pd.DataFrame(data=imp.transform(crime) , columns=crime.columns)

In [124]:
crime.dtypes

Population           float64
RapeLegacy           float64
Robbery              float64
AggravatedAssault    float64
Burglary             float64
LarcenyTheft         float64
MotorVehicleTheft    float64
Arson3               float64
dtype: object

In [125]:
for col in crime.columns:
    crime[col] = crime[col].apply(pd.to_numeric).astype('int64')

In [126]:
crime.dtypes

Population           int64
RapeLegacy           int64
Robbery              int64
AggravatedAssault    int64
Burglary             int64
LarcenyTheft         int64
MotorVehicleTheft    int64
Arson3               int64
dtype: object

In [127]:
crime.describe()

,Population,RapeLegacy,Robbery,AggravatedAssault,Burglary,LarcenyTheft,MotorVehicleTheft,Arson3
count,351.000,351.000,351.000,351.000,351.000,351.000,351.000,351.000
mean,40037.627,5.858,72.895,121.259,119.678,637.017,35.897,1.464
std,448104.485,60.166,1026.605,1698.804,920.976,6318.799,401.691,7.808
min,526.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,3010.500,0.000,0.000,1.000,6.000,31.000,0.000,0.000
50%,7411.000,0.000,1.000,5.000,18.000,95.000,2.000,1.000
75%,19324.500,2.000,5.000,14.500,52.500,290.000,7.000,1.000
max,8396126.000,1112.000,19170.000,31767.000,16606.000,117931.000,7434.000,132.000


In [128]:
crime.isnull().sum().sort_values(ascending=False).head()

Arson3               0
MotorVehicleTheft    0
LarcenyTheft         0
Burglary             0
AggravatedAssault    0
dtype: int64

In [129]:
crime.dtypes

Population           int64
RapeLegacy           int64
Robbery              int64
AggravatedAssault    int64
Burglary             int64
LarcenyTheft         int64
MotorVehicleTheft    int64
Arson3               int64
dtype: object

I've realized that PropertyCrime is the sum of Burglery, Larceny Theft, and Mother Vegicle Theft. So we're going to delete that and make a prediction for Burglary. I will continue to use the non-property crime data as that might be informative and I can still check the correlations for feature reduction if need be. 

In [130]:
X = crime.drop('Burglary', 1)
y = crime['Burglary']

In [131]:
def add_interactions(X):
    # Get feature names
    combos = list(combinations(list(X.columns), 2))
    colnames = list(X.columns) + ['_'.join(X) for X in combos]
    
    # Find interactions
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    X = poly.fit_transform(X)
    X = pd.DataFrame(X)
    X.columns = colnames
    
    # Remove interaction terms with all 0 values            
    noint_indicies = [i for i, X in enumerate(list((X == 0).all())) if X]
    X = X.drop(X.columns[noint_indicies], axis=1)
    
    return X

In [132]:
X = add_interactions(X)
print(X.head(5))

   Population  RapeLegacy  Robbery  AggravatedAssault  LarcenyTheft  \
0    1861.000       0.000    0.000              0.000        10.000   
1    2577.000       0.000    0.000              3.000        20.000   
2    2846.000       0.000    0.000              3.000        15.000   
3   97956.000      30.000  227.000            526.000      3243.000   
4    6388.000       3.000    4.000             16.000       165.000   

   MotorVehicleTheft  Arson3  Population_RapeLegacy  Population_Robbery  \
0              0.000   0.000                  0.000               0.000   
1              1.000   0.000                  0.000               0.000   
2              0.000   0.000                  0.000               0.000   
3            142.000   1.000            2938680.000        22236012.000   
4              5.000   1.000              19164.000           25552.000   

   Population_AggravatedAssault            ...             \
0                         0.000            ...              


In [133]:
pca = PCA(n_components=10)
X_pca = pd.DataFrame(pca.fit_transform(X))

In [134]:
X_pca.head()

,0,1,2,3,4,5,6,7,8,9
0,-2985194456.634,-2649528.820,740209.962,195136.484,25472.137,-29473.364,4146.583,-7594.834,-2380.855,-1460.825
1,-2985160965.062,-2647194.064,737686.414,191444.550,27268.525,-28878.363,4512.808,-7027.920,-2030.254,-1361.034
2,-2985169339.225,-2650988.499,737274.524,192258.615,27132.460,-29333.629,4400.966,-6770.347,-1892.781,-1278.389
3,-2665219963.143,31324285.784,-24481439.266,-8908971.890,863513.128,-334807.955,779792.703,71082.407,-35999.093,-22091.174
4,-2984176335.999,-2474992.840,613579.910,157506.779,31655.573,-37159.041,-1258.086,-3393.427,-1021.715,-606.709


In [135]:
y.head()

0      2
1      3
2      1
3    705
4     53
Name: Burglary, dtype: int64

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, train_size=0.70, random_state=1)

In [137]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [138]:
y_pred = regression_model.predict(X_test)

In [139]:
# The coefficients
print('Coefficients: \n', regression_model.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [ 1.59120127e-08  1.56440747e-05  1.34483326e-05 -3.82852003e-05
  3.69639368e-05 -4.69988328e-05 -8.89272399e-06  3.41564378e-03
 -1.55123316e-03  5.19833126e-03]
Mean squared error: 6678.36
Variance score: 0.90


In [140]:
crime.columns

Index(['Population', 'RapeLegacy', 'Robbery', 'AggravatedAssault', 'Burglary',
       'LarcenyTheft', 'MotorVehicleTheft', 'Arson3'],
      dtype='object')

In [141]:
linear_formula = 'Burglary ~ Population+RapeLegacy+Robbery+AggravatedAssault+LarcenyTheft+MotorVehicleTheft+Arson3'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=crime).fit()

In [142]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Burglary   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 3.666e+04
Date:                Thu, 17 May 2018   Prob (F-statistic):               0.00
Time:                        20:24:06   Log-Likelihood:                -1731.6
No. Observations:                 351   AIC:                             3479.
Df Residuals:                     343   BIC:                             3510.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             9.7093      2.275      4.268      0.000       5.235      14.183
Population           -0.0018      0.000    -11.176      0.000      -0.002      -0.001
RapeLegacy            6.2769      0.738      8.504      0.000       4.825       7.729
Robbery              -0.7324      0.152     -4.826      0.000      -1.031      -0.434
AggravatedAssault     0.0423      0.088      0.482      0.630      -0.130       0.215
LarcenyTheft          0.1350      0.010     13.703      0.000       0.116       0.154
MotorVehicleTheft     2.9008      0.153     18.975      0.000       2.600       3.201
Arson3                2.5065      0.310      8.074      0.000       1.896       3.117
==============================================================================
Omnibus:                      212.178   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3171.293
Skew:                           2.215   Prob(JB):                         0.00
Kurtosis:                      17.044   Cond. No.                     5.64e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.64e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [143]:
lm.params

Intercept            9.709
Population          -0.002
RapeLegacy           6.277
Robbery             -0.732
AggravatedAssault    0.042
LarcenyTheft         0.135
MotorVehicleTheft    2.901
Arson3               2.507
dtype: float64

It is clear that Population, Aggravated Assault, and Larceny Theft have little to no impact on Burglary. So I will re-run this linear formula without them. 

In [144]:
crime.head()

,Population,RapeLegacy,Robbery,AggravatedAssault,Burglary,LarcenyTheft,MotorVehicleTheft,Arson3
0,1861,0,0,0,2,10,0,0
1,2577,0,0,3,3,20,1,0
2,2846,0,0,3,1,15,0,0
3,97956,30,227,526,705,3243,142,1
4,6388,3,4,16,53,165,5,1


In [145]:
crime = crime.drop(['Population', 'AggravatedAssault', 'LarcenyTheft'], axis=1)
crime.head(10)

,RapeLegacy,Robbery,Burglary,MotorVehicleTheft,Arson3
0,0,0,2,0,0
1,0,0,3,1,0
2,0,0,1,0,0
3,30,227,705,142,1
4,3,4,53,5,1
5,0,3,10,0,1
6,0,0,0,0,0
7,7,31,204,32,3
8,2,4,16,6,1
9,0,12,99,15,0


Now that I have removed the low-impact features, let's expand what we have with reestablishing our interactions. An interesting thing happened when I tried to re-run the interactions without first redefining X and y. The interactions that already existed through the first run were then made as features for further interactions, creating hundreds (and then thousands) of features. So below I ahve redefined X and y. 

In [146]:
X = crime.drop('Burglary', 1)
y = crime['Burglary']

In [147]:
def add_interactions(X):
    # Get feature names
    combos = list(combinations(list(X.columns), 2))
    colnames = list(X.columns) + ['_'.join(X) for X in combos]
    
    # Find interactions
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    X = poly.fit_transform(X)
    X = pd.DataFrame(X)
    X.columns = colnames
    
    # Remove interaction terms with all 0 values            
    noint_indicies = [i for i, X in enumerate(list((X == 0).all())) if X]
    X = X.drop(X.columns[noint_indicies], axis=1)
    
    return X

In [148]:
X = add_interactions(X)
print(X.head(5))

   RapeLegacy  Robbery  MotorVehicleTheft  Arson3  RapeLegacy_Robbery  \
0       0.000    0.000              0.000   0.000               0.000   
1       0.000    0.000              1.000   0.000               0.000   
2       0.000    0.000              0.000   0.000               0.000   
3      30.000  227.000            142.000   1.000            6810.000   
4       3.000    4.000              5.000   1.000              12.000   

   RapeLegacy_MotorVehicleTheft  RapeLegacy_Arson3  Robbery_MotorVehicleTheft  \
0                         0.000              0.000                      0.000   
1                         0.000              0.000                      0.000   
2                         0.000              0.000                      0.000   
3                      4260.000             30.000                  32234.000   
4                        15.000              3.000                     20.000   

   Robbery_Arson3  MotorVehicleTheft_Arson3  
0           0.000           

In [149]:
pca = PCA(n_components=4)
X_pca = pd.DataFrame(pca.fit_transform(X))

Note that we are down to 4 featuires. This is the result of limitations of the svd_solver argument of the PCA decomposition.

In [150]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, train_size=0.70, random_state=1)

In [151]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [152]:
y_pred = regression_model.predict(X_test)

In [153]:
# The coefficients
print('Coefficients: \n', regression_model.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [ 0.00011484  0.03652587  0.04260175 -0.0088957 ]
Mean squared error: 64851.47
Variance score: 0.04


Our model still runs effectively at 92%. Let's try this entire process again for the 2014 data. I will make the following assumptions:

- The features that held little to no impact in 2013 will hold little to no impact in 2014 and will be therefor excluded
- The dataset will require the same cleaning as the 2013 data

In [166]:
crime2 = pd.read_csv('2014Offenses.csv',
                    header=0,
                    names=['City', 'Population', 'ViolentCrime', 'Manslaughter', 'RapeCurrent', 'RapeLegacy', 'Robbery', 'AggravatedAssault', 'PropertyCrime', 'Burglary', 'LarcenyTheft', 'MotorVehicleTheft', 'Arson3'],
                    )
crime2 = crime2.drop(['City', 'RapeLegacy', 'Manslaughter', 'PropertyCrime', 'ViolentCrime', 'RapeCurrent'], axis=1)
crime2.head(10)

,Population,Robbery,AggravatedAssault,Burglary,LarcenyTheft,MotorVehicleTheft,Arson3
0,"1,851",0,0,1,10,0,0.000
1,"2,568",1,1,1,47,1,0.000
2,820,0,0,0,1,0,0.000
3,"2,842",0,1,0,17,0,0.000
4,"98,595",237,503,683,"3,083",122,12.000
5,"5,872",2,21,41,159,4,0.000
6,"1,107",0,0,2,5,0,0.000
7,"4,032",0,9,6,24,0,0.000
8,"1,723",0,1,2,0,0,0.000
9,"118,860",43,68,176,"1,846",44,2.000


In [167]:
for col in crime2.columns[:-1]:
    crime2[col] = crime2[col].str.replace(",","")

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(crime2)
crime2 = pd.DataFrame(data=imp.transform(crime2) , columns=crime2.columns)

In [168]:
crime2.dtypes

Population           float64
Robbery              float64
AggravatedAssault    float64
Burglary             float64
LarcenyTheft         float64
MotorVehicleTheft    float64
Arson3               float64
dtype: object

In [169]:
for col in crime2.columns:
    crime2[col] = crime2[col].apply(pd.to_numeric).astype('int64')

In [170]:
crime2.isnull().sum().sort_values(ascending=False).head()

Arson3               0
MotorVehicleTheft    0
LarcenyTheft         0
Burglary             0
AggravatedAssault    0
dtype: int64

In [171]:
X = crime2.drop('Burglary', 1)
y = crime2['Burglary']

In [172]:
def add_interactions(X):
    # Get feature names
    combos = list(combinations(list(X.columns), 2))
    colnames = list(X.columns) + ['_'.join(X) for X in combos]
    
    # Find interactions
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    X = poly.fit_transform(X)
    X = pd.DataFrame(X)
    X.columns = colnames
    
    # Remove interaction terms with all 0 values            
    noint_indicies = [i for i, X in enumerate(list((X == 0).all())) if X]
    X = X.drop(X.columns[noint_indicies], axis=1)
    
    return X

In [173]:
X = add_interactions(X)
print(X.head(5))

   Population  Robbery  AggravatedAssault  LarcenyTheft  MotorVehicleTheft  \
0    1851.000    0.000              0.000        10.000              0.000   
1    2568.000    1.000              1.000        47.000              1.000   
2     820.000    0.000              0.000         1.000              0.000   
3    2842.000    0.000              1.000        17.000              0.000   
4   98595.000  237.000            503.000      3083.000            122.000   

   Arson3  Population_Robbery  Population_AggravatedAssault  \
0   0.000               0.000                         0.000   
1   0.000            2568.000                      2568.000   
2   0.000               0.000                         0.000   
3   0.000               0.000                      2842.000   
4  12.000        23367015.000                  49593285.000   

   Population_LarcenyTheft  Population_MotorVehicleTheft  \
0                18510.000                         0.000   
1               120696.000      

In [174]:
pca = PCA(n_components=10)
X_pca = pd.DataFrame(pca.fit_transform(X))

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, train_size=0.70, random_state=1)

In [176]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [177]:
y_pred = regression_model.predict(X_test)

In [178]:
# The coefficients
print('Coefficients: \n', regression_model.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [ 1.58885248e-08  1.32182441e-05  8.81648058e-06  5.75712943e-06
  1.03233431e-04 -1.01418673e-05  2.93831064e-03  9.85676110e-05
  7.44691553e-03 -9.42857668e-04]
Mean squared error: 1188.94
Variance score: 0.89


With an 89% variance score, this linear regression applies to two separate yars of data equivalently. 